In [1]:
import logging
import random

from RQ2.constants import LOGS_DIR, MODELS_DIR
from RQ1.scripts import run_model_on_vec_env
import numpy as np
from common.scripts import combine_df
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv

from common.env.forex_env import DiscreteActionForexEnv
from common.feature.feature_engineer import FeatureEngineer
from common.feature.stepwise_feature_engineer import StepwiseFeatureEngineer

from common.data import ForexData
from common.constants import *
from common.scripts import *

In [34]:
# Get ask and bid data, and combine
ask_path = FOREX_DIR / "EURUSD" / "15M" / "ASK" / "10.05.2022T00.00-10.05.2025T23.45.csv"
bid_path = FOREX_DIR / "EURUSD" / "15M" / "BID" / "10.05.2022T00.00-10.05.2025T23.45.csv"
ask_df = ForexData(ask_path).df
bid_df = ForexData(ask_path).df
forex_data = combine_df(bid_df, ask_df)
# forex_data = filter_df(forex_data)
train_df, eval_df = split_df(forex_data, 0.7)

train_df


,date_gmt,open_bid,high_bid,low_bid,close_bid,volume_bid,open_ask,high_ask,low_ask,close_ask,volume_ask,volume
0,2022-05-10 00:00:00+00:00,1.05635,1.05647,1.05559,1.05566,1.233490e+09,1.05635,1.05647,1.05559,1.05566,1.233490e+09,1.233490e+09
1,2022-05-10 00:15:00+00:00,1.05567,1.05625,1.05536,1.05610,2.519210e+09,1.05567,1.05625,1.05536,1.05610,2.519210e+09,2.519210e+09
2,2022-05-10 00:30:00+00:00,1.05609,1.05654,1.05590,1.05603,2.022950e+09,1.05609,1.05654,1.05590,1.05603,2.022950e+09,2.022950e+09
3,2022-05-10 00:45:00+00:00,1.05604,1.05644,1.05563,1.05591,2.623300e+09,1.05604,1.05644,1.05563,1.05591,2.623300e+09,2.623300e+09
4,2022-05-10 01:00:00+00:00,1.05591,1.05680,1.05555,1.05676,3.237470e+09,1.05591,1.05680,1.05555,1.05676,3.237470e+09,3.237470e+09
...,...,...,...,...,...,...,...,...,...,...,...,...
38648,2024-06-23 04:00:00+00:00,1.06939,1.06939,1.06939,1.06939,0.000000e+00,1.06939,1.06939,1.06939,1.06939,0.000000e+00,0.000000e+00
38649,2024-06-23 04:15:00+00:00,1.06939,1.06939,1.06939,1.06939,0.000000e+00,1.06939,1.06939,1.06939,1.06939,0.000000e+00,0.000000e+00
38650,2024-06-23 04:30:00+00:00,1.06939,1.06939,1.06939,1.06939,0.000000e+00,1.06939,1.06939,1.06939,1.06939,0.000000e+00,0.000000e+00
38651,2024-06-23 04:45:00+00:00,1.06939,1.06939,1.06939,1.06939,0.000000e+00,1.06939,1.06939,1.06939,1.06939,0.000000e+00,0.000000e+00


In [35]:
# drop everything except for closebid
train_df = train_df[["close_bid"]]

In [36]:
train_df['delta'] = train_df['close_bid'].diff()
train_df[370:390]

C:\Users\Finn\AppData\Local\Temp\ipykernel_12308\3363481552.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['delta'] = train_df['close_bid'].diff()


,close_bid,delta
370,1.04064,0.00030
371,1.04158,0.00094
372,1.04158,0.00000
373,1.04158,0.00000
374,1.04158,0.00000
375,1.04158,0.00000
376,1.04158,0.00000
377,1.04158,0.00000
378,1.04158,0.00000
379,1.04158,0.00000


In [38]:
train_df['gain'] = train_df['delta'].apply(lambda x: 1 if x > 0 else 0)
train_df['loss'] = train_df['delta'].apply(lambda x: 1 if x < 0 else 0)
train_df['gain'] = train_df['gain'].rolling(14).mean()
train_df['loss'] = train_df['loss'].rolling(14).mean()
train_df['rs'] = train_df['gain'] / train_df['loss']
train_df[370:390]

C:\Users\Finn\AppData\Local\Temp\ipykernel_12308\1385900896.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['gain'] = train_df['delta'].apply(lambda x: 1 if x > 0 else 0)
C:\Users\Finn\AppData\Local\Temp\ipykernel_12308\1385900896.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['loss'] = train_df['delta'].apply(lambda x: 1 if x < 0 else 0)
C:\Users\Finn\AppData\Local\Temp\ipykernel_12308\1385900896.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

,close_bid,delta,gain,loss,rs
370,1.04064,0.00030,0.428571,0.571429,0.750000
371,1.04158,0.00094,0.500000,0.500000,1.000000
372,1.04158,0.00000,0.500000,0.428571,1.166667
373,1.04158,0.00000,0.500000,0.357143,1.400000
374,1.04158,0.00000,0.500000,0.285714,1.750000
375,1.04158,0.00000,0.428571,0.285714,1.500000
376,1.04158,0.00000,0.357143,0.285714,1.250000
377,1.04158,0.00000,0.285714,0.285714,1.000000
378,1.04158,0.00000,0.285714,0.214286,1.333333
379,1.04158,0.00000,0.285714,0.142857,2.000000


In [25]:
# --- Feature Engineering ---
# Create a feature engineer object
feature_engineer = FeatureEngineer()

def rsi(df, window=14):
    """
    Calculate the Relative Strength Index (RSI) column.
    """
    delta = df['close_bid'].diff()
    gain = (delta.where(delta >= 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta <= 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    df['rsi'] = rsi
    return df

feature_engineer.add(rsi)

In [26]:
new_df = feature_engineer.run(train_df)

In [27]:
new_df[370:390]


,rsi
370,40.798226
371,56.389452
372,57.201646
373,72.020725
374,78.753541
375,76.780186
376,76.038339
377,70.930233
378,73.493976
379,87.980769
